**Import Libraries**

In [3]:
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



**Define Foursquare Credentials**

In [4]:
ClIENT_ID = 'Q0PR2R0PC5VKZ0BKORIBNO2JGRU4SOAWMYQWGKKK0ZSXVFLV' # your Foursquare ID
ClIENT_SECRET = 'YLCZCD1PNRXM5YL4QPFBDXN3SY3DQAUSJAQ0VYDFXQFU4LV0' # your Foursquare Secret
VERSION = '20180604'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: Q0PR2R0PC5VKZ0BKORIBNO2JGRU4SOAWMYQWGKKK0ZSXVFLV
Foursquare_Secret:YLCZCD1PNRXM5YL4QPFBDXN3SY3DQAUSJAQ0VYDFXQFU4LV0


**Define the city and get latitude and longitude**

In [5]:
city = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


**Search for hotels**

In [6]:
# search for hotels
search_query = 'Hotel'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=Q0PR2R0PC5VKZ0BKORIBNO2JGRU4SOAWMYQWGKKK0ZSXVFLV&client_secret=YLCZCD1PNRXM5YL4QPFBDXN3SY3DQAUSJAQ0VYDFXQFU4LV0&ll=43.6534817,-79.3839347&v=20180604&query=Hotel&radius=500&limit=30'

In [8]:
import requests # library to handle requests
# Send the GET Request and examine the results
results = requests.get(url).json()
#results

In [10]:

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,4ab2d511f964a5209b6c20e3,Sheraton Centre Toronto Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1597836635,False,123 Queen Street West,at York St.,43.650594,-79.384530,"[{'label': 'display', 'lat': 43.6505944, 'lng'...",324,M5H 2M9,CA,Toronto,ON,Canada,"[123 Queen Street West (at York St.), Toronto ...",NaN,NaN
1,4af96fbbf964a520c01122e3,One King West Hotel & Residence,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1597836635,False,1 King St W,at Yonge St.,43.649139,-79.377876,"[{'label': 'display', 'lat': 43.6491395, 'lng'...",686,M5H 1A1,CA,Toronto,ON,Canada,"[1 King St W (at Yonge St.), Toronto ON M5H 1A...",NaN,NaN
2,51d212c3498ebf27dc469bc9,Chelsea Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1597836635,False,33 Gerrard Street West,at Yonge St,43.658498,-79.383097,"[{'label': 'display', 'lat': 43.65849759157591...",562,M5G 1Z4,CA,Toronto,ON,Canada,"[33 Gerrard Street West (at Yonge St), Toronto...",NaN,NaN
3,4ae61cf6f964a520caa421e3,Pantages Hotel & Spa,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1597836635,False,200 Victoria St,at Shuter St,43.654498,-79.379035,"[{'label': 'display', 'lat': 43.65449797039222...",410,NaN,CA,Toronto,ON,Canada,"[200 Victoria St (at Shuter St), Toronto ON, C...",NaN,NaN
4,51e48697498eded9073c6c17,Crew Room Eaton Chelsea Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1597836635,False,33 Gerrard Street West,NaN,43.658094,-79.382711,"[{'label': 'display', 'lat': 43.65809435580944...",522,NaN,CA,Toronto,ON,Canada,"[33 Gerrard Street West, Toronto ON, Canada]",NaN,NaN


**Clean Hotel Dataframe**

In [11]:
# keep only columns that include venue name, and anything that is associated with location
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,at York St.,43.650594,-79.384530,"[{'label': 'display', 'lat': 43.6505944, 'lng'...",324,M5H 2M9,CA,Toronto,ON,Canada,"[123 Queen Street West (at York St.), Toronto ...",NaN,4ab2d511f964a5209b6c20e3
1,One King West Hotel & Residence,Hotel,1 King St W,at Yonge St.,43.649139,-79.377876,"[{'label': 'display', 'lat': 43.6491395, 'lng'...",686,M5H 1A1,CA,Toronto,ON,Canada,"[1 King St W (at Yonge St.), Toronto ON M5H 1A...",NaN,4af96fbbf964a520c01122e3
2,Chelsea Hotel,Hotel,33 Gerrard Street West,at Yonge St,43.658498,-79.383097,"[{'label': 'display', 'lat': 43.65849759157591...",562,M5G 1Z4,CA,Toronto,ON,Canada,"[33 Gerrard Street West (at Yonge St), Toronto...",NaN,51d212c3498ebf27dc469bc9
3,Pantages Hotel & Spa,Hotel,200 Victoria St,at Shuter St,43.654498,-79.379035,"[{'label': 'display', 'lat': 43.65449797039222...",410,NaN,CA,Toronto,ON,Canada,"[200 Victoria St (at Shuter St), Toronto ON, C...",NaN,4ae61cf6f964a520caa421e3
4,Crew Room Eaton Chelsea Hotel,Hotel,33 Gerrard Street West,NaN,43.658094,-79.382711,"[{'label': 'display', 'lat': 43.65809435580944...",522,NaN,CA,Toronto,ON,Canada,"[33 Gerrard Street West, Toronto ON, Canada]",NaN,51e48697498eded9073c6c17


In [12]:
# delete unnecessary columns
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
2,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
3,Pantages Hotel & Spa,Hotel,200 Victoria St,43.654498,-79.379035,NaN,ON
4,Crew Room Eaton Chelsea Hotel,Hotel,33 Gerrard Street West,43.658094,-79.382711,NaN,ON
5,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
6,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
8,Sheraton Centre Toronto Hotel - Grand Ballroom,Ballroom,123 Queen Street West,43.651200,-79.384520,NaN,ON
9,St Regis Hotel,Hotel,325 Bay St,43.649770,-79.380701,M5H 4G3,ON


In [26]:
# delete rows with none values
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False).reset_index(drop = True)


clean_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
2,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
3,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
4,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
5,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
6,St Regis Hotel,Hotel,325 Bay St,43.649770,-79.380701,M5H 4G3,ON
7,Pantages Hotel Gym,Gym,200 Victoria St.,43.654464,-79.379036,M5B 1V8,ON
8,Pantages Hotel Toronto Centre,Residential Building (Apartment / Condo),200 Victoria Street,43.655205,-79.379182,M5B 1V8,ON
9,The Adelaide Hotel Toronto,Hotel,325 Bay St,43.649831,-79.380164,M5H 4G3,ON


In [28]:

# delete rows which its category is not Hotel or Event Space
array= ['Hotel', 'Event Space']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)].reset_index(drop = True)
hotel_dataframe

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
2,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
3,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
4,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
5,St Regis Hotel,Hotel,325 Bay St,43.649770,-79.380701,M5H 4G3,ON
6,The Adelaide Hotel Toronto,Hotel,325 Bay St,43.649831,-79.380164,M5H 4G3,ON
7,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
8,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON
9,Hilton,Hotel,145 Richmond St W,43.649946,-79.385479,M5H 2L2,ON


In [30]:

# choose the hotel which has the same postalCode with the event space
df_hotel = hotel_dataframe[hotel_dataframe.postalCode == 'M5H 2M9']
df_hotel

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.38453,M5H 2M9,ON


**Search for Parks**

In [31]:
# search for Parks
search_query = 'Park'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=Q0PR2R0PC5VKZ0BKORIBNO2JGRU4SOAWMYQWGKKK0ZSXVFLV&client_secret=YLCZCD1PNRXM5YL4QPFBDXN3SY3DQAUSJAQ0VYDFXQFU4LV0&ll=43.6534817,-79.3839347&v=20180604&query=Park&radius=10000&limit=30'

In [32]:
# Send the GET Request and examine the results
presults = requests.get(url).json()
#presults

In [33]:
# assign relevant part of JSON to venues
venues = presults['response']['venues']

# tranform venues into a dataframe
park_dataframe = json_normalize(venues)
park_dataframe.head()

C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.crossStreet,location.postalCode
0,4b54deadf964a520a3d027e3,Grange Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1597838627,False,43.652488,-79.392053,"[{'label': 'display', 'lat': 43.65248755299856...",663,CA,Toronto,ON,Canada,"[btwn College, University, Queen & Spadina, To...",NaN,NaN,NaN
1,4b170e47f964a5208cc123e3,Queen's Park Subway Station,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",v-1597838627,False,43.660006,-79.390272,"[{'label': 'display', 'lat': 43.66000635700919...",887,CA,Toronto,ON,Canada,"[691 University Ave (at College St), Toronto O...",691 University Ave,at College St,M5G 2P1
2,4bfc5019da7120a132df49fd,St. Andrews Playground / Dog Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1597838627,False,43.647388,-79.398752,"[{'label': 'display', 'lat': 43.64738759338116...",1372,CA,Toronto,ON,Canada,[450 Adelaide Street West (Brant St & Adelaide...,450 Adelaide Street West,Brant St & Adelaide St W,NaN
3,4b9d206bf964a520e69136e3,Queen's Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1597838627,False,43.663946,-79.392180,"[{'label': 'display', 'lat': 43.66394609897775...",1340,CA,Toronto,ON,Canada,"[University Ave. (at Wellesley Ave.), Toronto ...",University Ave.,at Wellesley Ave.,M5R 2E8
4,4bb79860cf2fc9b616779e02,Berczy Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1597838627,False,43.648048,-79.375172,"[{'label': 'display', 'lat': 43.64804835388851...",929,CA,Toronto,ON,Canada,"[35 Wellington St. East, Toronto ON, Canada]",35 Wellington St. East,NaN,NaN


**Clean the Park Dataframe**

In [34]:
# keep only columns that include venue name, and anything that is associated with location
park_clean_columns = ['name', 'categories'] + [col for col in park_dataframe.columns if col.startswith('location.')]+ ['id']
clean_park_dataframe = park_dataframe.loc[:,park_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_park_dataframe['categories'] = clean_park_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_park_dataframe.columns = [column.split('.')[-1] for column in clean_park_dataframe.columns]

clean_park_dataframe.head()

,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,crossStreet,postalCode,id
0,Grange Park,Park,43.652488,-79.392053,"[{'label': 'display', 'lat': 43.65248755299856...",663,CA,Toronto,ON,Canada,"[btwn College, University, Queen & Spadina, To...",NaN,NaN,NaN,4b54deadf964a520a3d027e3
1,Queen's Park Subway Station,Metro Station,43.660006,-79.390272,"[{'label': 'display', 'lat': 43.66000635700919...",887,CA,Toronto,ON,Canada,"[691 University Ave (at College St), Toronto O...",691 University Ave,at College St,M5G 2P1,4b170e47f964a5208cc123e3
2,St. Andrews Playground / Dog Park,Park,43.647388,-79.398752,"[{'label': 'display', 'lat': 43.64738759338116...",1372,CA,Toronto,ON,Canada,[450 Adelaide Street West (Brant St & Adelaide...,450 Adelaide Street West,Brant St & Adelaide St W,NaN,4bfc5019da7120a132df49fd
3,Queen's Park,Park,43.663946,-79.392180,"[{'label': 'display', 'lat': 43.66394609897775...",1340,CA,Toronto,ON,Canada,"[University Ave. (at Wellesley Ave.), Toronto ...",University Ave.,at Wellesley Ave.,M5R 2E8,4b9d206bf964a520e69136e3
4,Berczy Park,Park,43.648048,-79.375172,"[{'label': 'display', 'lat': 43.64804835388851...",929,CA,Toronto,ON,Canada,"[35 Wellington St. East, Toronto ON, Canada]",35 Wellington St. East,NaN,NaN,4bb79860cf2fc9b616779e02


In [35]:

# delete unnecessary columns
clean_park_dataframe2= clean_park_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_park_dataframe2

,name,categories,lat,lng,state,address,postalCode
0,Grange Park,Park,43.652488,-79.392053,ON,NaN,NaN
1,Queen's Park Subway Station,Metro Station,43.660006,-79.390272,ON,691 University Ave,M5G 2P1
2,St. Andrews Playground / Dog Park,Park,43.647388,-79.398752,ON,450 Adelaide Street West,NaN
3,Queen's Park,Park,43.663946,-79.392180,ON,University Ave.,M5R 2E8
4,Berczy Park,Park,43.648048,-79.375172,ON,35 Wellington St. East,NaN
5,TTC Streetcar #403 - Victoria park,Light Rail Station,43.650735,-79.386921,ON,NaN,NaN
6,Clarence Square Park,Park,43.644222,-79.394230,ON,Spadina Ave,NaN
7,One Park Lane,Residential Building (Apartment / Condo),43.655442,-79.390056,ON,280 Simcoe St,M5T 2Y5
8,Trinity Bellwoods Park,Park,43.647072,-79.413756,ON,1053 Dundas St. W.,M5H 2N2
9,Physiomed College Park,Medical Center,43.659240,-79.382860,ON,382 Yonge St,M5B 1S8


In [36]:
# delete rows with none values
clean_park_dataframe3 = clean_park_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False).reset_index(drop=True)
clean_park_dataframe3

,name,categories,lat,lng,state,address,postalCode
0,Queen's Park Subway Station,Metro Station,43.660006,-79.390272,ON,691 University Ave,M5G 2P1
1,Queen's Park,Park,43.663946,-79.392180,ON,University Ave.,M5R 2E8
2,One Park Lane,Residential Building (Apartment / Condo),43.655442,-79.390056,ON,280 Simcoe St,M5T 2Y5
3,Trinity Bellwoods Park,Park,43.647072,-79.413756,ON,1053 Dundas St. W.,M5H 2N2
4,Physiomed College Park,Medical Center,43.659240,-79.382860,ON,382 Yonge St,M5B 1S8
5,Canoe Landing Park,Park,43.638762,-79.397067,ON,50 Fort York Blvd,M5V 3Z1
6,Residences at College Park North,Residential Building (Apartment / Condo),43.659822,-79.385159,ON,763 Bay St.,M5G 2R3
7,Bellevue Square Park,Park,43.653610,-79.402199,ON,btwn Bellevue & Augusta Ave.,M5T 2N4
8,Riverdale Park East,Park,43.669951,-79.355493,ON,550 Broadview Ave,M4K 2P1
9,Riverdale Park West,Park,43.666048,-79.360941,ON,500 Gerrard St.,M5A 2H3


In [38]:

# delete rows which its category is not Park
df_park = clean_park_dataframe3[clean_park_dataframe3.categories == 'Park'].reset_index(drop=True)
df_park

,name,categories,lat,lng,state,address,postalCode
0,Queen's Park,Park,43.663946,-79.392180,ON,University Ave.,M5R 2E8
1,Trinity Bellwoods Park,Park,43.647072,-79.413756,ON,1053 Dundas St. W.,M5H 2N2
2,Canoe Landing Park,Park,43.638762,-79.397067,ON,50 Fort York Blvd,M5V 3Z1
3,Bellevue Square Park,Park,43.653610,-79.402199,ON,btwn Bellevue & Augusta Ave.,M5T 2N4
4,Riverdale Park East,Park,43.669951,-79.355493,ON,550 Broadview Ave,M4K 2P1
5,Riverdale Park West,Park,43.666048,-79.360941,ON,500 Gerrard St.,M5A 2H3
6,High Park,Park,43.646479,-79.463425,ON,1873 Bloor St. W,M6R 2Z3


**Search For Restaurants**

In [41]:
# search for Restaurants
search_query = 'Restaurant'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=Q0PR2R0PC5VKZ0BKORIBNO2JGRU4SOAWMYQWGKKK0ZSXVFLV&client_secret=YLCZCD1PNRXM5YL4QPFBDXN3SY3DQAUSJAQ0VYDFXQFU4LV0&ll=43.6534817,-79.3839347&v=20180604&query=Restaurant&radius=10000&limit=30'

In [40]:

# Send the GET Request and examine the results
Rresults = requests.get(url).json()
#Rresults

In [42]:
# assign relevant part of JSON to venues
venues = Rresults['response']['venues']

# tranform venues into a dataframe
Restaurant_dataframe = json_normalize(venues)
Restaurant_dataframe.head()

C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id,location.neighborhood
0,4ad4c05ff964a52048f720e3,Hemispheres Restaurant & Bistro,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1597845976,False,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",224,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,NaN
1,4ad4c05cf964a52006f620e3,Victoria's Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1597845976,False,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",763,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,498556908,NaN
2,4ada5d5bf964a520e92121e3,The Hot House Restaurant & Bar,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1597845976,False,35 Church St,43.648824,-79.373702,"[{'label': 'display', 'lat': 43.64882370529773...",973,M5E 1T3,CA,Toronto,ON,Canada,"[35 Church St (at Front St E), Toronto ON M5E ...",at Front St E,NaN,NaN
3,5750b013498e755287c6de97,Some Time BBQ Grill Restaurant 碳烤屋,"[{'id': '52af3b773cf9994f4e043c03', 'name': 'S...",v-1597845976,False,988 Baldwin Street,43.655874,-79.393826,"[{'label': 'display', 'lat': 43.655874, 'lng':...",839,NaN,CA,Toronto,ON,Canada,"[988 Baldwin Street, Toronto ON, Canada]",NaN,NaN,NaN
4,4ad4c060f964a5207ff720e3,Rol San Restaurant 龍笙棧,"[{'id': '4bf58dd8d48988d1f5931735', 'name': 'D...",v-1597845976,False,323 Spadina Ave.,43.654318,-79.398650,"[{'label': 'display', 'lat': 43.65431754076345...",1188,M5T 2E9,CA,Toronto,ON,Canada,"[323 Spadina Ave. (at D'Arcy St.), Toronto ON ...",at D'Arcy St.,NaN,Kensington Market


**Clean Restaurant Dataframe**

In [43]:
# keep only columns that include venue name, and anything that is associated with location
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]

clean_Restaurant_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",224,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,4ad4c05ff964a52048f720e3
1,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",763,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,NaN,4ad4c05cf964a52006f620e3
2,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,"[{'label': 'display', 'lat': 43.64882370529773...",973,M5E 1T3,CA,Toronto,ON,Canada,"[35 Church St (at Front St E), Toronto ON M5E ...",at Front St E,NaN,4ada5d5bf964a520e92121e3
3,Some Time BBQ Grill Restaurant 碳烤屋,Szechuan Restaurant,988 Baldwin Street,43.655874,-79.393826,"[{'label': 'display', 'lat': 43.655874, 'lng':...",839,NaN,CA,Toronto,ON,Canada,"[988 Baldwin Street, Toronto ON, Canada]",NaN,NaN,5750b013498e755287c6de97
4,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,"[{'label': 'display', 'lat': 43.65431754076345...",1188,M5T 2E9,CA,Toronto,ON,Canada,"[323 Spadina Ave. (at D'Arcy St.), Toronto ON ...",at D'Arcy St.,Kensington Market,4ad4c060f964a5207ff720e3


In [44]:

# delete unnecessary columns
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Restaurant_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
2,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,M5E 1T3,ON
3,Some Time BBQ Grill Restaurant 碳烤屋,Szechuan Restaurant,988 Baldwin Street,43.655874,-79.393826,NaN,ON
4,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,M5T 2E9,ON
5,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,M5V 2X3,ON
6,Tasty's Caribbean Restaurant & Catering,Caribbean Restaurant,405 Spadina Ave,43.656794,-79.399251,M5T 2G6,ON
7,Studio Restaurant,Breakfast Spot,389 Church St.,43.661500,-79.379319,M5B,ON
8,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,43.652278,-79.398039,M5T 2E4,ON
9,Victor Restaurant & Bar,Bar,30 Mercer Street,43.645634,-79.391125,M5V 1H3,ON


In [45]:
# delete rows with none values
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False).reset_index(drop=True)
df_Restaurant

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
2,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,M5E 1T3,ON
3,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,M5T 2E9,ON
4,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,M5V 2X3,ON
5,Tasty's Caribbean Restaurant & Catering,Caribbean Restaurant,405 Spadina Ave,43.656794,-79.399251,M5T 2G6,ON
6,Studio Restaurant,Breakfast Spot,389 Church St.,43.661500,-79.379319,M5B,ON
7,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,43.652278,-79.398039,M5T 2E4,ON
8,Victor Restaurant & Bar,Bar,30 Mercer Street,43.645634,-79.391125,M5V 1H3,ON
9,Aroma Fine Indian Restaurant,Indian Restaurant,287 King St. W,43.646463,-79.389644,M5V 1J5,ON


**Search For Cafeteria**

In [46]:
# search for Cafeteria
search_query = 'Cafeteria'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=Q0PR2R0PC5VKZ0BKORIBNO2JGRU4SOAWMYQWGKKK0ZSXVFLV&client_secret=YLCZCD1PNRXM5YL4QPFBDXN3SY3DQAUSJAQ0VYDFXQFU4LV0&ll=43.6534817,-79.3839347&v=20180604&query=Cafeteria&radius=10000&limit=30'

In [47]:

# Send the GET Request and examine the results
cresults = requests.get(url).json()
#cresults

In [48]:
# assign relevant part of JSON to venues
venues = cresults['response']['venues']

# tranform venues into a dataframe
Cafeteria_dataframe = json_normalize(venues)
Cafeteria_dataframe.head()

C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode
0,4f9165bde4b04fef0087e4e2,Cafeteria,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",v-1597847483,False,The Hospital for Sick Children (SickKids),43.657209,-79.386063,"[{'label': 'display', 'lat': 43.65720930549997...",448,CA,Toronto,ON,Canada,"[The Hospital for Sick Children (SickKids), To...",NaN,NaN
1,528e99a211d262edc3708a6f,Stay Cafeteria 慢走,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",v-1597847483,False,388 Spadina Ave,43.655454,-79.399163,"[{'label': 'display', 'lat': 43.65545427446646...",1246,CA,Toronto,ON,Canada,"[388 Spadina Ave (at Nassau St), Toronto ON M5...",at Nassau St,M5T 2G5
2,4b57256bf964a520332828e3,Sears Cafeteria,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",v-1597847483,False,Yonge,43.656038,-79.380672,"[{'label': 'display', 'lat': 43.65603785909391...",387,CA,Toronto,ON,Canada,"[Yonge (Dundas), Toronto ON, Canada]",Dundas,NaN
3,53bd6dfe498e6c1d3bc9333e,Industrial Alliance Cafeteria,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",v-1597847483,False,NaN,43.655657,-79.389443,"[{'label': 'display', 'lat': 43.655657, 'lng':...",505,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN
4,524f1c8f498e983dde61ada0,Chestnut Tree Cafeteria,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",v-1597847483,False,89 Chestnut Street,43.648231,-79.384045,"[{'label': 'display', 'lat': 43.648231, 'lng':...",584,CA,Toronto,ON,Canada,"[89 Chestnut Street, Toronto ON, Canada]",NaN,NaN


**Clean Cafeteria Dataframe**

In [49]:
# keep only columns that include venue name, and anything that is associated with location
Cafeteria_clean_columns = ['name', 'categories'] + [col for col in Cafeteria_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Cafeteria_dataframe = Cafeteria_dataframe.loc[:,Cafeteria_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']

# filter the category for each row
clean_Cafeteria_dataframe['categories'] = clean_Cafeteria_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Cafeteria_dataframe.columns = [column.split('.')[-1] for column in clean_Cafeteria_dataframe.columns]

clean_Cafeteria_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,id
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),43.657209,-79.386063,"[{'label': 'display', 'lat': 43.65720930549997...",448,CA,Toronto,ON,Canada,"[The Hospital for Sick Children (SickKids), To...",NaN,NaN,4f9165bde4b04fef0087e4e2
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,"[{'label': 'display', 'lat': 43.65545427446646...",1246,CA,Toronto,ON,Canada,"[388 Spadina Ave (at Nassau St), Toronto ON M5...",at Nassau St,M5T 2G5,528e99a211d262edc3708a6f
2,Sears Cafeteria,Food Court,Yonge,43.656038,-79.380672,"[{'label': 'display', 'lat': 43.65603785909391...",387,CA,Toronto,ON,Canada,"[Yonge (Dundas), Toronto ON, Canada]",Dundas,NaN,4b57256bf964a520332828e3
3,Industrial Alliance Cafeteria,Cafeteria,NaN,43.655657,-79.389443,"[{'label': 'display', 'lat': 43.655657, 'lng':...",505,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN,53bd6dfe498e6c1d3bc9333e
4,Chestnut Tree Cafeteria,Cafeteria,89 Chestnut Street,43.648231,-79.384045,"[{'label': 'display', 'lat': 43.648231, 'lng':...",584,CA,Toronto,ON,Canada,"[89 Chestnut Street, Toronto ON, Canada]",NaN,NaN,524f1c8f498e983dde61ada0


In [50]:
# delete unnecessary columns
clean_Cafeteria_dataframe2= clean_Cafeteria_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Cafeteria_dataframe2

,name,categories,address,lat,lng,state,postalCode
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),43.657209,-79.386063,ON,NaN
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,ON,M5T 2G5
2,Sears Cafeteria,Food Court,Yonge,43.656038,-79.380672,ON,NaN
3,Industrial Alliance Cafeteria,Cafeteria,NaN,43.655657,-79.389443,ON,NaN
4,Chestnut Tree Cafeteria,Cafeteria,89 Chestnut Street,43.648231,-79.384045,ON,NaN
5,OCAD Cafeteria,College Cafeteria,100 McCaul St.,43.652780,-79.391762,ON,NaN
6,Dears Cafeteria,Dessert Shop,NaN,43.654179,-79.392555,NaN,NaN
7,Ryerson Hub Cafeteria,College Cafeteria,350 Victoria St.,43.658475,-79.377653,ON,NaN
8,SickKids Cafeteria,Food Court,555 University Avenue,43.657584,-79.375652,ON,NaN
9,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,ON,M4V 1H7


In [52]:
# delete rows with none values
df_Cafeteria = clean_Cafeteria_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False).reset_index(drop=True)
df_Cafeteria

,name,categories,address,lat,lng,state,postalCode
0,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,ON,M5T 2G5
1,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,ON,M4V 1H7
2,Marketeria,Restaurant,30 Bond St.,43.653585,-79.378843,ON,M5B 1W8
3,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,ON,M5B 2K3
4,Innis College,Student Center,2 Sussex Ave,43.665556,-79.399298,ON,M5S 1J5
5,6th Floor Cafeteria 12 Concorde Place,Café,12 Concorde Place,43.721861,-79.329339,ON,m3c 3k7


**Search For Shopping Stores**

In [55]:
# search for Shopping
search_query = 'Shopping'
radius = 1000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=Q0PR2R0PC5VKZ0BKORIBNO2JGRU4SOAWMYQWGKKK0ZSXVFLV&client_secret=YLCZCD1PNRXM5YL4QPFBDXN3SY3DQAUSJAQ0VYDFXQFU4LV0&ll=43.6534817,-79.3839347&v=20180604&query=Shopping&radius=1000&limit=30'

In [54]:
# Send the GET Request and examine the results
sresults = requests.get(url).json()
#sresults

In [56]:
# assign relevant part of JSON to venues
venues = sresults['response']['venues']

# tranform venues into a dataframe
Shopping_dataframe = json_normalize(venues)
Shopping_dataframe.head()

C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.distance,location.postalCode,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress,location.labeledLatLngs
0,4ad77a12f964a520260b21e3,CF Toronto Eaton Centre,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1597847768,False,220 Yonge St,btwn Queen & Dundas,43.654540,-79.380677,287,M5B 2H1,CA,"Downtown Toronto, Toronto, ON",Toronto,ON,Canada,"[220 Yonge St (btwn Queen & Dundas), Toronto O...",NaN
1,5840cafe06f1a34af80cc609,Saks Fifth Avenue Club - Personal Shopping,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",v-1597847768,False,176 Yonge Street,Queen Street West,43.651810,-79.379192,424,M5C 2L7,CA,NaN,Toronto,ON,Canada,"[176 Yonge Street (Queen Street West), Toronto...","[{'label': 'display', 'lat': 43.65181, 'lng': ..."
2,4b9e86dff964a520e6eb36e3,TD Centre Shopping Concourse,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1597847768,False,66 Wellington St W,btw York & Bay St,43.647184,-79.380932,741,M5K 1A1,CA,NaN,Toronto,ON,Canada,"[66 Wellington St W (btw York & Bay St), Toron...","[{'label': 'display', 'lat': 43.64718424611984..."


**Clean Shopping Dataframe**

In [57]:
# keep only columns that include venue name, and anything that is associated with location
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']

# filter the category for each row
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]

clean_Shopping_dataframe.head()

,name,categories,address,crossStreet,lat,lng,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,labeledLatLngs,id
0,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,btwn Queen & Dundas,43.654540,-79.380677,287,M5B 2H1,CA,"Downtown Toronto, Toronto, ON",Toronto,ON,Canada,"[220 Yonge St (btwn Queen & Dundas), Toronto O...",NaN,4ad77a12f964a520260b21e3
1,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,Queen Street West,43.651810,-79.379192,424,M5C 2L7,CA,NaN,Toronto,ON,Canada,"[176 Yonge Street (Queen Street West), Toronto...","[{'label': 'display', 'lat': 43.65181, 'lng': ...",5840cafe06f1a34af80cc609
2,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,btw York & Bay St,43.647184,-79.380932,741,M5K 1A1,CA,NaN,Toronto,ON,Canada,"[66 Wellington St W (btw York & Bay St), Toron...","[{'label': 'display', 'lat': 43.64718424611984...",4b9e86dff964a520e6eb36e3


In [58]:
# delete unnecessary columns
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Shopping_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON
1,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,43.651810,-79.379192,M5C 2L7,ON
2,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON


In [60]:
# delete rows which its category is not Shopping Mall
df_Shopping = clean_Shopping_dataframe2[clean_Shopping_dataframe2.categories == 'Shopping Mall'].reset_index(drop=True)
df_Shopping

,name,categories,address,lat,lng,postalCode,state
0,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON
1,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON


**Generate map to visualize hotels, shopping stores and Cafeteria and how they cluster together**

In [62]:
# create dataframe of hotels, shopping stores and Cafeteria
Final_df1 = pd.concat([df_hotel, df_Cafeteria, df_Shopping], ignore_index=True)
Final_df1

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
2,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON
3,Marketeria,Restaurant,30 Bond St.,43.653585,-79.378843,M5B 1W8,ON
4,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,M5B 2K3,ON
5,Innis College,Student Center,2 Sussex Ave,43.665556,-79.399298,M5S 1J5,ON
6,6th Floor Cafeteria 12 Concorde Place,Café,12 Concorde Place,43.721861,-79.329339,m3c 3k7,ON
7,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON
8,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON


In [63]:
# Generate map to visualize hotel neighbourhood including shopping stores and Cafeteria 
Final1_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(Final_df1['lat'], Final_df1['lng'], 
                                           Final_df1['name'], Final_df1['categories'],\
                                               Final_df1['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(Final1_map)  
    
Final1_map

**Generate map to visualize Park, Restaurant and Cafeteria and how they cluster together**

In [64]:
# create dataframe of Park, Restaurant and Cafeteria
Final_df2 = pd.concat([df_park, df_Restaurant, df_Cafeteria,], ignore_index=True)
Final_df2

,name,categories,lat,lng,state,address,postalCode
0,Queen's Park,Park,43.663946,-79.392180,ON,University Ave.,M5R 2E8
1,Trinity Bellwoods Park,Park,43.647072,-79.413756,ON,1053 Dundas St. W.,M5H 2N2
2,Canoe Landing Park,Park,43.638762,-79.397067,ON,50 Fort York Blvd,M5V 3Z1
3,Bellevue Square Park,Park,43.653610,-79.402199,ON,btwn Bellevue & Augusta Ave.,M5T 2N4
4,Riverdale Park East,Park,43.669951,-79.355493,ON,550 Broadview Ave,M4K 2P1
5,Riverdale Park West,Park,43.666048,-79.360941,ON,500 Gerrard St.,M5A 2H3
6,High Park,Park,43.646479,-79.463425,ON,1873 Bloor St. W,M6R 2Z3
7,Hemispheres Restaurant & Bistro,American Restaurant,43.654884,-79.385931,ON,110 Chestnut Street,M5G 1R3
8,Victoria's Restaurant,Restaurant,43.649298,-79.376431,ON,37 King Street East,M5C 1E9
9,The Hot House Restaurant & Bar,American Restaurant,43.648824,-79.373702,ON,35 Church St,M5E 1T3


In [65]:
# Generate map to visualize park neighbourhood including Restaurant and Cafeteria 
Final2_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(Final_df2['lat'], Final_df2['lng'], 
                                           Final_df2['name'], Final_df2['categories'],\
                                               Final_df2['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(Final2_map)  
    
Final2_map